In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
target_path = '/content/drive/MyDrive/Customer Segmentation in US/notebooks'

# 3️⃣ توليد اسم الملف مع التاريخ والوقت
import datetime
ts = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
target_filename = f'3_Modeling.ipynb'

# 4️⃣ جلب محتوى النوتبوك الحالي
from google.colab import _message
nb_json = _message.blocking_request('get_ipynb', '')['ipynb']

# 5️⃣ حفظ الملف داخل الفولدر المحدد
import os, json
os.makedirs(target_path, exist_ok=True)
save_path = os.path.join(target_path, target_filename)
with open(save_path, 'w', encoding='utf-8') as f:
    f.write(json.dumps(nb_json))

print('✅ Notebook saved successfully at:\n', save_path)